In [1]:
import api
import datasets
import pandas as pd

from models import CommandConfiguration
from models import CorrectTaskResult

/Users/koissiadjorlolo/Desktop/OI/oi-integration-tests/gaia/py/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base = "http://192.168.67.3:7172"
# base = "http://127.0.0.1:7172"

In [3]:
print(f"is_connected: {api.check_connection(base)}")

is_connected: False


In [8]:
data = datasets.load_dataset("gaia-benchmark/GAIA", "2023_all", split="validation")
df = pd.DataFrame(data)
level1 = data[:4]
level1

{'task_id': ['c61d22de-5f6c-4958-a7f6-5e9707bd3466',
  '17b5a6a3-bc87-42e8-b0fb-6ab0781ef2cc',
  '04a04a9b-226c-43fd-b319-d5e89743676f',
  '14569e28-c88c-43e4-8c32-097d35b9a67d'],
 'Question': ['A paper about AI regulation that was originally submitted to arXiv.org in June 2022 shows a figure with three axes, where each axis has a label word at both ends. Which of these words is used to describe a type of society in a Physics and Society article submitted to arXiv.org on August 11, 2016?',
  'I’m researching species that became invasive after people who kept them as pets released them. There’s a certain species of fish that was popularized as a pet by being the main character of the movie Finding Nemo. According to the USGS, where was this fish found as a nonnative species, before the year 2020? I need the answer formatted as the five-digit zip codes of the places the species was found, separated by commas if there is more than one place.',
  'If we assume all articles published by Nat

In [5]:
command = CommandConfiguration(
    auto_run=True,
    os_mode=False,
    model="ollama/llama3",
    api_base="http://192.168.1.86:11434",
    api_key="",
    system_prompt="""You are a general AI assistant.
I will ask you a question. Report your thoughts, and finish your answer with the following template: FINAL ANSWER: [YOUR FINAL ANSWER].
YOUR FINAL ANSWER should be a number OR as few words as possible OR a comma separated list of numbers and/or strings.
If you are asked for a number, don’t use comma to write your number neither use units such as $ or percent sign unless specified otherwise.
If you are asked for a string, don’t use articles, neither abbreviations (e.g. for cities), and write the digits in plain text unless specified otherwise.
If you are asked for a comma separated list, apply the above rules depending of whether the element to be put in the list is a number or a string."""
)

In [7]:
# api.run_single(base, command, level1[0])
from gaia.py.models import CorrectTaskResult


print("running", len(level1), "tasks.")
n_so_far = 0
n_passing = 0
for task_id in level1:
    try:
        result = api.run_single(base, command, task_id, timeout_s=20 * 60)
        print(f"{n_so_far} / {len(level1)}:", result)
        n_so_far += 1
        n_passing += 1 if isinstance(result, CorrectTaskResult) else 0
    except Exception as e:
        print(e)
        raise e

print(f"{n_passing} / {len(level1)} passed.")

status='not-found' created=datetime.datetime(2024, 5, 16, 22, 52, 13, 738513) conversation=[{'role': 'assistant', 'type': 'message', 'content': "**Plan Recap:** None yet, let's start planning!\n\nTo solve this problem, I will first calculate the distance between the Earth and the Moon at its closest approach (called perigee). Then, I'll divide Eliud Kipchoge's marathon pace by a conversion factor to get his speed in kilometers per hour. Finally, I'll use this speed to calculate how many hours it would take him to run the calculated distance.\n\n**Code Block:**\n"}, {'role': 'assistant', 'type': 'code', 'format': 'python', 'content': '\nimport computer\n\n# Calculate the distance between Earth and Moon at its closest approach (perigee)\nperigee_distance = 356400 # km (minimum value from Wikipedia)\n\n# Convert Eliud Kipchoge\'s marathon pace to kilometers per hour\nmarathon_pace = 2.27 # hours (record-breaking pace)\nspeed_kmh = marathon_pace * 1000 / 42.195 # convert hours to km/h\n\np

ReadTimeout: HTTPConnectionPool(host='192.168.67.3', port=7172): Read timed out. (read timeout=1200)

In [10]:
gen = api.check_runs(base)
watching = set()

for task_id in level1["task_id"]:
    run_id = api.invoke(base, command, task_id)
    if run_id is not None:
        watching.add(run_id)

for msg in gen:
    if len(watching) == 0:
        break
    print("message:", msg)

KeyboardInterrupt: 